# Question 6.1

In [14]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, merge, concatenate, Maximum
from keras.optimizers import SGD
from keras import regularizers
import numpy as np

batch_size = 128
num_classes = 10


# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [15]:
#Defining expand grid function 
import itertools
import pandas as pd

def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [17]:
# Creating training grid
trainGrid = expand_grid({ 'regularization_l2':[1e-1,1e-2,1e-3,1e-4],
                          'step_size': [1e-1,1e-2,1e-3],
                         'epoch':[10,15],
                        #create a column to store accuracies in
                          'Validation_accuracy': [0.0],
                         
                        })
trainGrid['regularization_l2']= trainGrid['regularization_l2'].apply(lambda x: float(x))
trainGrid

,regularization_l2,step_size,epoch,Validation_accuracy
0,0.1000,0.100,10,0.0
1,0.1000,0.100,15,0.0
2,0.1000,0.010,10,0.0
3,0.1000,0.010,15,0.0
4,0.1000,0.001,10,0.0
5,0.1000,0.001,15,0.0
6,0.0100,0.100,10,0.0
7,0.0100,0.100,15,0.0
8,0.0100,0.010,10,0.0
9,0.0100,0.010,15,0.0


In [18]:
from sklearn.model_selection import KFold

for idx,item in trainGrid.iterrows():
    print(item)
    kfold = KFold(3)
    validation_accuracy=[]
    l2=item['regularization_l2']
    stepsize=item['step_size'] 
    Accuracy=[]
    epoch=int(item['epoch']) 
    model = Sequential()
    model.add(Dense(512, activation='linear',kernel_regularizer=regularizers.l2(l2), input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(l2)))
    
    sgd = SGD(lr=stepsize, decay=1e-6, momentum=0.9, nesterov=True)
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
    
    for k,(train,test) in enumerate(kfold.split(x_train,y_train)):
        history = model.fit(x_train[train], y_train[train],
                            batch_size=batch_size,
                            epochs=epoch,
                            verbose=1,
                            validation_data=(x_train[test], y_train[test]))
        score = model.evaluate(x_train[test], y_train[test], verbose=0)   
        Accuracy.append(score[1])
     
    print(item)
    trainGrid.loc[idx,'Validation_accuracy']=np.mean(Accuracy)
    print("Accuracy: %0.2f" % (trainGrid.loc[idx,'Validation_accuracy']))
        
        

regularization_l2       0.1
step_size               0.1
epoch                  10.0
Validation_accuracy     0.0
Name: 0, dtype: float64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


40000/40000 [==============================] - 12s 291us/step - loss: 2.2303 - acc: 0.5598 - val_loss: 1.8924 - val_acc: 0.5760
Epoch 2/10
40000/40000 [==============================] - 11s 283us/step - loss: 2.0625 - acc: 0.5743 - val_loss: 2.4927 - val_acc: 0.5361
Epoch 3/10
40000/40000 [==============================] - 11s 270us/step - loss: 2.2806 - acc: 0.5350 - val_loss: 1.6295 - val_acc: 0.6872
Epoch 4/10
40000/40000 [==============================] - 11s 283us/step - loss: 2.1416 - acc: 0.5534 - val_loss: 2.6873 - val_acc: 0.4350
Epoch 5/10
40000/40000 [==============================] - 11s 286us/step - loss: 2.2559 - acc: 0.5405 - val_loss: 1.8126 - val_acc: 0.5877
Epoch 6/10
40000/40000 [==============================] - 11s 286us/step - loss: 2.3106 - acc: 0.5220 - val_loss: 2.4969 - val_acc: 0.2767
Epoch 7/10
40000/40000 [==============================] - 11s 285us/step - loss: 2.3687 - acc: 0.4782 - val_loss: 2.3243 - val_acc: 0.6387
Epoch 8/10
40000/40000 [==============

Epoch 8/10
40000/40000 [==============================] - 11s 270us/step - loss: 1.3663 - acc: 0.8410 - val_loss: 1.3325 - val_acc: 0.8480
Epoch 9/10
40000/40000 [==============================] - 11s 271us/step - loss: 1.3362 - acc: 0.8407 - val_loss: 1.3057 - val_acc: 0.8499
Epoch 10/10
40000/40000 [==============================] - 11s 276us/step - loss: 1.3117 - acc: 0.8447 - val_loss: 1.2830 - val_acc: 0.8525
Train on 40000 samples, validate on 20000 samples
Epoch 1/10
40000/40000 [==============================] - 12s 302us/step - loss: 1.2865 - acc: 0.8459 - val_loss: 1.2679 - val_acc: 0.8532
Epoch 2/10
40000/40000 [==============================] - 12s 303us/step - loss: 1.2664 - acc: 0.8480 - val_loss: 1.2516 - val_acc: 0.8545
Epoch 3/10
40000/40000 [==============================] - 12s 299us/step - loss: 1.2502 - acc: 0.8476 - val_loss: 1.2356 - val_acc: 0.8541
Epoch 4/10
40000/40000 [==============================] - 13s 315us/step - loss: 1.2322 - acc: 0.8510 - val_loss: 1

Epoch 6/10
40000/40000 [==============================] - 13s 327us/step - loss: 1.1106 - acc: 0.8604 - val_loss: 1.0800 - val_acc: 0.8736
Epoch 7/10
40000/40000 [==============================] - 13s 337us/step - loss: 1.1044 - acc: 0.8595 - val_loss: 1.0728 - val_acc: 0.8720
Epoch 8/10
40000/40000 [==============================] - 13s 336us/step - loss: 1.0966 - acc: 0.8606 - val_loss: 1.0661 - val_acc: 0.8715
Epoch 9/10
40000/40000 [==============================] - 13s 314us/step - loss: 1.0894 - acc: 0.8595 - val_loss: 1.0602 - val_acc: 0.8724
Epoch 10/10
40000/40000 [==============================] - 13s 332us/step - loss: 1.0835 - acc: 0.8601 - val_loss: 1.0531 - val_acc: 0.8737
regularization_l2       0.100
step_size               0.001
epoch                  15.000
Validation_accuracy     0.000
Name: 5, dtype: float64
Accuracy: 0.86
regularization_l2       0.01
step_size               0.10
epoch                  10.00
Validation_accuracy     0.00
Name: 6, dtype: float64
_____

Epoch 5/10
40000/40000 [==============================] - 11s 277us/step - loss: 0.5753 - acc: 0.9007 - val_loss: 0.5394 - val_acc: 0.9159
Epoch 6/10
40000/40000 [==============================] - 12s 293us/step - loss: 0.5706 - acc: 0.9016 - val_loss: 0.5366 - val_acc: 0.9150
Epoch 7/10
40000/40000 [==============================] - 12s 301us/step - loss: 0.5631 - acc: 0.9037 - val_loss: 0.5365 - val_acc: 0.9149
Epoch 8/10
40000/40000 [==============================] - 11s 287us/step - loss: 0.5680 - acc: 0.9056 - val_loss: 0.5963 - val_acc: 0.8938
Epoch 9/10
40000/40000 [==============================] - 11s 287us/step - loss: 0.5581 - acc: 0.9084 - val_loss: 0.4960 - val_acc: 0.9287
Epoch 10/10
40000/40000 [==============================] - 12s 298us/step - loss: 0.5613 - acc: 0.9079 - val_loss: 0.5565 - val_acc: 0.9106
Train on 40000 samples, validate on 20000 samples
Epoch 1/10
40000/40000 [==============================] - 12s 291us/step - loss: 0.5640 - acc: 0.9084 - val_loss: 0

...

40000/40000 [==============================] - 10s 243us/step - loss: 1.8525 - acc: 0.6739 - val_loss: 1.2435 - val_acc: 0.8520
Epoch 2/10
40000/40000 [==============================] - 8s 200us/step - loss: 1.1936 - acc: 0.8469 - val_loss: 1.0720 - val_acc: 0.8815
Epoch 3/10
40000/40000 [==============================] - 9s 235us/step - loss: 1.0802 - acc: 0.8727 - val_loss: 1.0099 - val_acc: 0.8950
Epoch 4/10
40000/40000 [==============================] - 9s 234us/step - loss: 1.0278 - acc: 0.8838 - val_loss: 0.9716 - val_acc: 0.9014
Epoch 5/10
40000/40000 [==============================] - 9s 213us/step - loss: 0.9910 - acc: 0.8922 - val_loss: 0.9449 - val_acc: 0.9069
Epoch 6/10
40000/40000 [==============================] - 9s 217us/step - loss: 0.9642 - acc: 0.8998 - val_loss: 0.9214 - val_acc: 0.9105
Epoch 7/10
40000/40000 [==============================] - 8s 200us/step - loss: 0.9404 - acc: 0.9036 - val_loss: 0.9022 - val_acc: 0.9143
Epoch 8/10
40000/40000 [====================

Train on 40000 samples, validate on 20000 samples
Epoch 1/10
40000/40000 [==============================] - 8s 205us/step - loss: 0.3692 - acc: 0.9067 - val_loss: 0.2425 - val_acc: 0.9469
Epoch 2/10
40000/40000 [==============================] - 8s 191us/step - loss: 0.2384 - acc: 0.9489 - val_loss: 0.2189 - val_acc: 0.9529
Epoch 3/10
40000/40000 [==============================] - 8s 189us/step - loss: 0.2068 - acc: 0.9575 - val_loss: 0.1999 - val_acc: 0.9620
Epoch 4/10
40000/40000 [==============================] - 8s 188us/step - loss: 0.1964 - acc: 0.9613 - val_loss: 0.2054 - val_acc: 0.9631
Epoch 5/10
40000/40000 [==============================] - 8s 188us/step - loss: 0.1891 - acc: 0.9627 - val_loss: 0.2055 - val_acc: 0.9630
Epoch 6/10
40000/40000 [==============================] - 7s 187us/step - loss: 0.1841 - acc: 0.9651 - val_loss: 0.1796 - val_acc: 0.9705
Epoch 7/10
40000/40000 [==============================] - 8s 204us/step - loss: 0.1717 - acc: 0.9684 - val_loss: 0.1844 - 

40000/40000 [==============================] - 9s 216us/step - loss: 0.1234 - acc: 0.9819 - val_loss: 0.1282 - val_acc: 0.9802
Epoch 10/10
40000/40000 [==============================] - 8s 199us/step - loss: 0.1213 - acc: 0.9825 - val_loss: 0.1294 - val_acc: 0.9793
Train on 40000 samples, validate on 20000 samples
Epoch 1/10
40000/40000 [==============================] - 8s 196us/step - loss: 0.1317 - acc: 0.9784 - val_loss: 0.1024 - val_acc: 0.9877
Epoch 2/10
40000/40000 [==============================] - 8s 192us/step - loss: 0.1256 - acc: 0.9809 - val_loss: 0.1008 - val_acc: 0.9886
Epoch 3/10
40000/40000 [==============================] - 8s 191us/step - loss: 0.1205 - acc: 0.9818 - val_loss: 0.1026 - val_acc: 0.9875
Epoch 4/10
40000/40000 [==============================] - 8s 189us/step - loss: 0.1176 - acc: 0.9827 - val_loss: 0.1032 - val_acc: 0.9866
Epoch 5/10
40000/40000 [==============================] - 10s 238us/step - loss: 0.1155 - acc: 0.9838 - val_loss: 0.1017 - val_acc: 

Train on 40000 samples, validate on 20000 samples
Epoch 1/10
40000/40000 [==============================] - 8s 203us/step - loss: 1.2197 - acc: 0.6904 - val_loss: 0.6502 - val_acc: 0.8531
Epoch 2/10
40000/40000 [==============================] - 7s 181us/step - loss: 0.6054 - acc: 0.8497 - val_loss: 0.4934 - val_acc: 0.8833
Epoch 3/10
40000/40000 [==============================] - 7s 182us/step - loss: 0.5066 - acc: 0.8729 - val_loss: 0.4369 - val_acc: 0.8960
Epoch 4/10
40000/40000 [==============================] - 7s 184us/step - loss: 0.4569 - acc: 0.8876 - val_loss: 0.4067 - val_acc: 0.9031
Epoch 5/10
40000/40000 [==============================] - 7s 183us/step - loss: 0.4278 - acc: 0.8947 - val_loss: 0.3851 - val_acc: 0.9091
Epoch 6/10
40000/40000 [==============================] - 7s 186us/step - loss: 0.4081 - acc: 0.8992 - val_loss: 0.3692 - val_acc: 0.9133
Epoch 7/10
40000/40000 [==============================] - 7s 181us/step - loss: 0.3899 - acc: 0.9056 - val_loss: 0.3556 - 

Epoch 9/10
40000/40000 [==============================] - 9s 219us/step - loss: 0.2852 - acc: 0.9367 - val_loss: 0.2730 - val_acc: 0.9399
Epoch 10/10
40000/40000 [==============================] - 9s 225us/step - loss: 0.2802 - acc: 0.9379 - val_loss: 0.2683 - val_acc: 0.9414
Train on 40000 samples, validate on 20000 samples
Epoch 1/10
40000/40000 [==============================] - 9s 229us/step - loss: 0.2847 - acc: 0.9370 - val_loss: 0.2468 - val_acc: 0.9486
Epoch 2/10
40000/40000 [==============================] - 10s 238us/step - loss: 0.2782 - acc: 0.9389 - val_loss: 0.2438 - val_acc: 0.9488
Epoch 3/10
40000/40000 [==============================] - 9s 234us/step - loss: 0.2726 - acc: 0.9401 - val_loss: 0.2415 - val_acc: 0.9493
Epoch 4/10
40000/40000 [==============================] - 11s 282us/step - loss: 0.2680 - acc: 0.9409 - val_loss: 0.2386 - val_acc: 0.9502
Epoch 5/10
40000/40000 [==============================] - 8s 212us/step - loss: 0.2630 - acc: 0.9427 - val_loss: 0.2361

In [19]:
trainGrid

,regularization_l2,step_size,epoch,Validation_accuracy
0,0.1000,0.100,10,0.485217
1,0.1000,0.100,15,0.520667
2,0.1000,0.010,10,0.864000
3,0.1000,0.010,15,0.870767
4,0.1000,0.001,10,0.860450
5,0.1000,0.001,15,0.862783
6,0.0100,0.100,10,0.901217
7,0.0100,0.100,15,0.910650
8,0.0100,0.010,10,0.931583
9,0.0100,0.010,15,0.931317


In [20]:
epoch=10
model = Sequential()
model.add(Dense(512, activation='linear',kernel_regularizer=regularizers.l2(1e-4), input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(1e-4)))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epoch,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_88 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_59 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_60 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 13s 209us/step - loss: 0.492

# Question 6.2

In [84]:
from __future__ import print_function

import keras
from keras.datasets import cifar10

batch_size = 128

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.reshape(50000, 3072)
x_test = x_test.reshape(10000, 3072)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes=len(np.unique(y_train))

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


50000 train samples
10000 test samples


In [85]:
#Defining expand grid function 
import itertools
import pandas as pd

def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())


In [86]:
# Creating training grid
trainGrid_cifar = expand_grid({ 'regularization_l2':[1e-3,1e-4],
                          'step_size': [1e-2,1e-3,1e-4],
                         'epoch':[20,30],
                        #create a column to store accuracies in
                          'Validation_accuracy': [0.0],
                         
                        })
trainGrid_cifar['regularization_l2']= trainGrid_cifar['regularization_l2'].apply(lambda x: float(x))
trainGrid_cifar

,regularization_l2,step_size,epoch,Validation_accuracy
0,0.0010,0.0100,20,0.0
1,0.0010,0.0100,30,0.0
2,0.0010,0.0010,20,0.0
3,0.0010,0.0010,30,0.0
4,0.0010,0.0001,20,0.0
5,0.0010,0.0001,30,0.0
6,0.0001,0.0100,20,0.0
7,0.0001,0.0100,30,0.0
8,0.0001,0.0010,20,0.0
9,0.0001,0.0010,30,0.0


In [88]:
for idx,item in trainGrid_cifar.iterrows():
    print(item)
    kfold = KFold(5)
    validation_accuracy=[]
    l2=item['regularization_l2']
    stepsize=item['step_size'] 
    Accuracy=[]
    epoch=int(item['epoch']) 
    model = Sequential()
    model.add(Dense(800, activation='linear',kernel_regularizer=regularizers.l2(l2), input_shape=(3072,)))
    model.add(Dropout(0.2))
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(l2)))
    
    sgd = SGD(lr=stepsize, decay=1e-6, momentum=0.9, nesterov=True)
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
    
    for k,(train,test) in enumerate(kfold.split(x_train,y_train)):
        history = model.fit(x_train[train], y_train[train],
                            batch_size=batch_size,
                            epochs=epoch,
                            verbose=1,
                            validation_data=(x_train[test], y_train[test]))
        score = model.evaluate(x_train[test], y_train[test], verbose=0)   
        Accuracy.append(score[1])
     
    print(item)
    trainGrid_cifar.loc[idx,'Validation_accuracy']=np.mean(Accuracy)
    print("Accuracy: %0.2f" % (trainGrid_cifar.loc[idx,'Validation_accuracy']))
        

regularization_l2       0.001
step_size               0.010
epoch                  20.000
Validation_accuracy     0.000
Name: 0, dtype: float64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_238 (Dense)            (None, 800)               2458400   
_________________________________________________________________
dropout_159 (Dropout)        (None, 800)               0         
_________________________________________________________________
dense_239 (Dense)            (None, 800)               640800    
_________________________________________________________________
dropout_160 (Dropout)        (None, 800)               0         
_________________________________________________________________
dense_240 (Dense)            (None, 10)                8010      
Total params: 3,107,210
Trainable params: 3,107,210
Non-trainable params: 0
______________________________________________________

40000/40000 [==============================] - 33s 825us/step - loss: 1.3673 - acc: 0.5727 - val_loss: 1.3838 - val_acc: 0.5576
Epoch 11/20
40000/40000 [==============================] - 33s 837us/step - loss: 1.3622 - acc: 0.5737 - val_loss: 1.4006 - val_acc: 0.5530
Epoch 12/20
40000/40000 [==============================] - 29s 737us/step - loss: 1.3585 - acc: 0.5758 - val_loss: 1.3828 - val_acc: 0.5601
Epoch 13/20
40000/40000 [==============================] - 30s 753us/step - loss: 1.3564 - acc: 0.5752 - val_loss: 1.4059 - val_acc: 0.5591
Epoch 14/20
40000/40000 [==============================] - 28s 702us/step - loss: 1.3577 - acc: 0.5762 - val_loss: 1.3773 - val_acc: 0.5657
Epoch 15/20
40000/40000 [==============================] - 28s 696us/step - loss: 1.3530 - acc: 0.5788 - val_loss: 1.3642 - val_acc: 0.5710
Epoch 16/20
40000/40000 [==============================] - 28s 695us/step - loss: 1.3477 - acc: 0.5790 - val_loss: 1.3948 - val_acc: 0.5604
Epoch 17/20
40000/40000 [=======

Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 32s 795us/step - loss: 3.1051 - acc: 0.3243 - val_loss: 2.8647 - val_acc: 0.3842
Epoch 2/30
40000/40000 [==============================] - 28s 707us/step - loss: 2.7868 - acc: 0.3953 - val_loss: 2.6380 - val_acc: 0.4319
Epoch 3/30
40000/40000 [==============================] - 28s 709us/step - loss: 2.5973 - acc: 0.4228 - val_loss: 2.4520 - val_acc: 0.4571
Epoch 4/30
40000/40000 [==============================] - 32s 796us/step - loss: 2.4434 - acc: 0.4420 - val_loss: 2.3794 - val_acc: 0.4426
Epoch 5/30
40000/40000 [==============================] - 31s 781us/step - loss: 2.3179 - acc: 0.4522 - val_loss: 2.2429 - val_acc: 0.4601
Epoch 6/30
40000/40000 [==============================] - 28s 711us/step - loss: 2.2065 - acc: 0.4661 - val_loss: 2.1544 - val_acc: 0.4724
Epoch 7/30
40000/40000 [==============================] - 29s 714us/step - loss: 2.1140 - acc: 0.4734 - val_loss: 2.

Epoch 29/30
40000/40000 [==============================] - 28s 708us/step - loss: 1.3294 - acc: 0.5878 - val_loss: 1.4676 - val_acc: 0.5526
Epoch 30/30
40000/40000 [==============================] - 28s 711us/step - loss: 1.3281 - acc: 0.5890 - val_loss: 1.4796 - val_acc: 0.5470
Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 28s 711us/step - loss: 1.3969 - acc: 0.5675 - val_loss: 1.2304 - val_acc: 0.6227
Epoch 2/30
40000/40000 [==============================] - 28s 710us/step - loss: 1.3769 - acc: 0.5739 - val_loss: 1.2621 - val_acc: 0.6080
Epoch 3/30
40000/40000 [==============================] - 29s 713us/step - loss: 1.3719 - acc: 0.5716 - val_loss: 1.2450 - val_acc: 0.6110
Epoch 4/30
40000/40000 [==============================] - 28s 711us/step - loss: 1.3590 - acc: 0.5781 - val_loss: 1.2821 - val_acc: 0.6038
Epoch 5/30
40000/40000 [==============================] - 29s 730us/step - loss: 1.3560 - acc: 0.5768 - val_loss: 

Epoch 27/30
40000/40000 [==============================] - 28s 709us/step - loss: 1.2567 - acc: 0.6231 - val_loss: 1.3981 - val_acc: 0.5828
Epoch 28/30
40000/40000 [==============================] - 28s 712us/step - loss: 1.2588 - acc: 0.6238 - val_loss: 1.4666 - val_acc: 0.5607
Epoch 29/30
40000/40000 [==============================] - 28s 705us/step - loss: 1.2590 - acc: 0.6239 - val_loss: 1.3754 - val_acc: 0.5929
Epoch 30/30
40000/40000 [==============================] - 28s 709us/step - loss: 1.2516 - acc: 0.6261 - val_loss: 1.4156 - val_acc: 0.5737
Train on 40000 samples, validate on 10000 samples
Epoch 1/30
40000/40000 [==============================] - 28s 706us/step - loss: 1.3312 - acc: 0.6006 - val_loss: 1.1820 - val_acc: 0.6532
Epoch 2/30
40000/40000 [==============================] - 28s 712us/step - loss: 1.3120 - acc: 0.6055 - val_loss: 1.2315 - val_acc: 0.6426
Epoch 3/30
40000/40000 [==============================] - 29s 727us/step - loss: 1.3026 - acc: 0.6069 - val_loss

...

40000/40000 [==============================] - 29s 724us/step - loss: 1.0973 - acc: 0.6660 - val_loss: 1.0757 - val_acc: 0.6800
Epoch 23/30
40000/40000 [==============================] - 29s 714us/step - loss: 1.0929 - acc: 0.6687 - val_loss: 1.0707 - val_acc: 0.6840
Epoch 24/30
40000/40000 [==============================] - 29s 721us/step - loss: 1.0855 - acc: 0.6690 - val_loss: 1.0743 - val_acc: 0.6811
Epoch 25/30
40000/40000 [==============================] - 29s 713us/step - loss: 1.0834 - acc: 0.6666 - val_loss: 1.0834 - val_acc: 0.6778
Epoch 26/30
40000/40000 [==============================] - 28s 712us/step - loss: 1.0802 - acc: 0.6721 - val_loss: 1.0793 - val_acc: 0.6794
Epoch 27/30
40000/40000 [==============================] - 29s 713us/step - loss: 1.0792 - acc: 0.6725 - val_loss: 1.0918 - val_acc: 0.6735
Epoch 28/30
40000/40000 [==============================] - 28s 709us/step - loss: 1.0754 - acc: 0.6720 - val_loss: 1.0814 - val_acc: 0.6794
Epoch 29/30
40000/40000 [=======

40000/40000 [==============================] - 28s 711us/step - loss: 1.8023 - acc: 0.4191 - val_loss: 1.7888 - val_acc: 0.4246
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 29s 713us/step - loss: 1.8123 - acc: 0.4135 - val_loss: 1.7346 - val_acc: 0.4453
Epoch 2/20
40000/40000 [==============================] - 28s 708us/step - loss: 1.8074 - acc: 0.4150 - val_loss: 1.7317 - val_acc: 0.4453
Epoch 3/20
40000/40000 [==============================] - 28s 712us/step - loss: 1.8015 - acc: 0.4176 - val_loss: 1.7304 - val_acc: 0.4436
Epoch 4/20
40000/40000 [==============================] - 28s 709us/step - loss: 1.8009 - acc: 0.4184 - val_loss: 1.7273 - val_acc: 0.4450
Epoch 5/20
40000/40000 [==============================] - 29s 714us/step - loss: 1.7960 - acc: 0.4193 - val_loss: 1.7253 - val_acc: 0.4483
Epoch 6/20
40000/40000 [==============================] - 29s 714us/step - loss: 1.7937 - acc: 0.4217 - val_loss: 1.7230 - val_

Epoch 18/20
40000/40000 [==============================] - 29s 730us/step - loss: 1.6680 - acc: 0.4667 - val_loss: 1.6487 - val_acc: 0.4767
Epoch 19/20
40000/40000 [==============================] - 29s 714us/step - loss: 1.6667 - acc: 0.4657 - val_loss: 1.6468 - val_acc: 0.4784
Epoch 20/20
40000/40000 [==============================] - 29s 715us/step - loss: 1.6643 - acc: 0.4678 - val_loss: 1.6472 - val_acc: 0.4782
regularization_l2       0.0001
step_size               0.0001
epoch                  20.0000
Validation_accuracy     0.0000
Name: 10, dtype: float64
Accuracy: 0.45
regularization_l2       0.0001
step_size               0.0001
epoch                  30.0000
Validation_accuracy     0.0000
Name: 11, dtype: float64
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_271 (Dense)            (None, 800)               2458400   
_________________________________________________________________
dro

40000/40000 [==============================] - 29s 729us/step - loss: 1.7809 - acc: 0.4244 - val_loss: 1.7689 - val_acc: 0.4326
Epoch 17/30
40000/40000 [==============================] - 28s 709us/step - loss: 1.7763 - acc: 0.4265 - val_loss: 1.7648 - val_acc: 0.4345
Epoch 18/30
40000/40000 [==============================] - 28s 708us/step - loss: 1.7746 - acc: 0.4261 - val_loss: 1.7638 - val_acc: 0.4343
Epoch 19/30
40000/40000 [==============================] - 28s 705us/step - loss: 1.7697 - acc: 0.4274 - val_loss: 1.7615 - val_acc: 0.4376
Epoch 20/30
40000/40000 [==============================] - 28s 710us/step - loss: 1.7671 - acc: 0.4313 - val_loss: 1.7579 - val_acc: 0.4361
Epoch 21/30
40000/40000 [==============================] - 28s 712us/step - loss: 1.7640 - acc: 0.4335 - val_loss: 1.7551 - val_acc: 0.4376
Epoch 22/30
40000/40000 [==============================] - 28s 711us/step - loss: 1.7605 - acc: 0.4320 - val_loss: 1.7529 - val_acc: 0.4371
Epoch 23/30
40000/40000 [=======

40000/40000 [==============================] - 29s 731us/step - loss: 1.6583 - acc: 0.4692 - val_loss: 1.6268 - val_acc: 0.4891
Epoch 15/30
40000/40000 [==============================] - 28s 708us/step - loss: 1.6571 - acc: 0.4688 - val_loss: 1.6261 - val_acc: 0.4904
Epoch 16/30
40000/40000 [==============================] - 28s 709us/step - loss: 1.6565 - acc: 0.4676 - val_loss: 1.6262 - val_acc: 0.4923
Epoch 17/30
40000/40000 [==============================] - 28s 704us/step - loss: 1.6535 - acc: 0.4675 - val_loss: 1.6253 - val_acc: 0.4895
Epoch 18/30
40000/40000 [==============================] - 29s 716us/step - loss: 1.6511 - acc: 0.4742 - val_loss: 1.6234 - val_acc: 0.4924
Epoch 19/30
40000/40000 [==============================] - 28s 709us/step - loss: 1.6503 - acc: 0.4716 - val_loss: 1.6230 - val_acc: 0.4912
Epoch 20/30
40000/40000 [==============================] - 29s 715us/step - loss: 1.6480 - acc: 0.4726 - val_loss: 1.6210 - val_acc: 0.4946
Epoch 21/30
40000/40000 [=======

In [89]:
trainGrid_cifar

,regularization_l2,step_size,epoch,Validation_accuracy
0,0.0010,0.0100,20,0.55212
1,0.0010,0.0100,30,0.55972
2,0.0010,0.0010,20,0.54994
3,0.0010,0.0010,30,0.57228
4,0.0010,0.0001,20,0.44410
5,0.0010,0.0001,30,0.46154
6,0.0001,0.0100,20,0.61436
7,0.0001,0.0100,30,0.64294
8,0.0001,0.0010,20,0.56700
9,0.0001,0.0010,30,0.60252


In [90]:
epoch=30
model = Sequential()
model.add(Dense(800, activation='linear',kernel_regularizer=regularizers.l2(1e-4), input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(800, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(1e-4)))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.2, nesterov=True)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epoch,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_274 (Dense)            (None, 800)               2458400   
_________________________________________________________________
dropout_183 (Dropout)        (None, 800)               0         
_________________________________________________________________
dense_275 (Dense)            (None, 800)               640800    
_________________________________________________________________
dropout_184 (Dropout)        (None, 800)               0         
_________________________________________________________________
dense_276 (Dense)            (None, 10)                8010      
Total params: 3,107,210
Trainable params: 3,107,210
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 41s 812us/step - loss: 2